In [1]:
%pylab notebook

Populating the interactive namespace from numpy and matplotlib


In [2]:
from numpy.random import rand
from numpy import prod

In [3]:
class layer(object):
    
    def __init__(self,shape):
        self.weights = rand(shape[0],shape[1]+1)
        
    def __call__(self,inputs):
        return dot(self.weights,append(inputs,[1]))
        

In [4]:
class network(object):
    
    def __init__(self,layer_sizes):
        self.layers=[]
        for i,s in enumerate(zip(layer_sizes,layer_sizes[1:])):
            self.layers.append(layer(s[::-1]))
            
    def threshold(self,x):
        return 1/(1+exp(-x))
    
    def threshold_der(self,x):
        return self.threshold(x)*(1-self.threshold(x))
    
    def __call__(self,inputs):
        return self.layer_output(inputs,-1)
    
    def layer_output(self,inputs,layer_num):
        #layers zero-indexed
        for l in self.layers[:layer_num]:
            inputs=self.threshold(l(inputs))
        return self.layers[layer_num](inputs)
    
    def get_err(self, inputs, target):
        output=self.__call__(inputs)
        return output-target
    
    def get_batches(self,data,batch_size):
        for i in range(0, len(data), batch_size):
            yield data[i:i + batch_size]
        
        
    
    def backpropagation(self,training_set,learning_curve,batch_size=1):
                
        batches=self.get_batches(training_set,batch_size)
        
        for batch in batches:
            for i,layer in enumerate(self.layers[::-1]):
                average_correction=0
                for t in batch:
                    inputs=t[0]
                    if i+1==len(self.layers):
                        layer_input=append(inputs,[1])
                    else:
                        layer_input=append(self.threshold(self.layer_output(inputs,len(self.layers)-(i+2))),[1]) #a^l-1
                    
                    if i ==0:
                        delta = -learning_curve*self.get_err(inputs,t[1])
                    else:
                        delta *= self.threshold_der(dot(layer.weights,layer_input))
                    average_correction+=outer(delta,layer_input)
                layer.weights+=average_correction/batch_size
                delta = dot(delta,layer.weights[:,:-1])
        
            

In [5]:
n=network([2,4,1])



In [6]:
test_set=[(x,prod(x)) for x in rand(10000,2)]
data,labels=zip(*test_set)

In [7]:
j=0
while (sum(abs(array([i[0] for i in map(n,data)])-array(labels))/labels)/10000)>0.01:
    n.backpropagation(test_set,.1,batch_size=1)
    j+=1
#     print(n(ones(4)))

KeyboardInterrupt: 

In [8]:
i =int(rand(1)*10000)
test_set[i][0], n(test_set[i][0]), test_set[i][1]

(array([ 0.33036917,  0.28542452]), array([ 0.03390279]), 0.094295459399586987)

In [9]:
(sum(abs(array([i[0] for i in map(n,data)])-array(labels))/labels)/10000)

3.8875539697354697

In [342]:
a1=n.layers[0].weights
a2=n.layers[1].weights

In [343]:
a1,a2

(array([[ 1.44656785,  0.95797093, -3.26703838],
        [ 1.70388876, -0.17189786,  0.57147739],
        [ 1.47574478,  1.05095545,  0.81501109],
        [-0.85892531,  1.34601507,  1.14245989]]),
 array([[ 3.10007786,  2.90466768, -2.73652221,  2.64274336, -2.08039467]]))

In [347]:
dot(a1,array([ 0.73195221,  0.2591262,1 ]))

array([-1.95998447,  1.77409929,  2.16751583,  0.86255538])

In [346]:
n.threshold(dot(a1,array([ 0.73195221,  0.2591262,1 ])))

array([ 0.12346873,  0.85496672,  0.89729426,  0.70319427])

In [349]:
dot(a2,append(n.threshold(dot(a1,array([ 0.73195221,  0.2591262,1 ]))),[1]))

array([ 0.1886585])

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation


model = Sequential([
    Dense(4, input_shape=(2,)),
    Activation('sigmoid'),
    Dense(1),
])

model.compile(optimizer='sgd',
              loss='mse')

model.fit(array(data), array(labels), epochs=1000, batch_size=10000)

Using TensorFlow backend.


KeyboardInterrupt: 